In [10]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv
import os
import configparser
from tensorflow.keras.preprocessing import image as imagekeras
import time

In [11]:
def predictimg(face_segment, model):
    face_resized = np.resize(face_segment, (48, 48, 3))
    X = imagekeras.img_to_array(face_resized)
    X = np.expand_dims(X, axis = 0)
    image_stk = np.vstack([X])
    return model.predict(image_stk) #yhat

In [12]:
FACE_CLASSIFIER = cv.CascadeClassifier(
    cv.data.haarcascades + "haarcascade_frontalface_default.xml"
)
PWD = os.path.abspath(os.getcwd())
CONFIG = configparser.RawConfigParser()
CONFIG.read('..\\..\\config\\.properties')
MODELS = CONFIG.get('GeneralConfiguration', 'models.dir');
MODEL = tf.keras.models.load_model(f'{MODELS}\\model1.h5')
CLASS_INDICES = {
    0:'Angry',
    1:'Disgusted',
    2:'Fearful',
    3:'Happy',
    4:'Neutral',
    5:'Sad',
    6:'Surprised'
}

In [13]:
vid = cv.VideoCapture(0)
tik = time.time()
while(True):
    ret, frame = vid.read()
    if ret == False:
        print('Error reading from camera')
        break
    frame_cpy = frame.copy()
    font                   = cv.FONT_HERSHEY_SIMPLEX
    pos                    = (10, 30)     # top left corner
    fontScale              = 1
    fontColor              = (0,0,255)   
    thickness              = 3
    lineType               = 2
    
    # OpenCV's face recognition
    gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    face = FACE_CLASSIFIER.detectMultiScale(
        gray_img, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40)
    )
    
    if len(face) == 0: # Fail safe for non-detection
        cv.putText(frame, '----', 
        pos, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType
        )
        
        cv.imshow('cv_recognition_model1', frame)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        continue
        
    for (x, y, w, h) in face:
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
    
    # Model prediction
    
    face_x = face[0][0]
    face_y = face[0][1]
    face_w = face[0][2]
    face_h = face[0][3]
    face_segment = frame_cpy[y:y+h, x:x+w, :]
        
    y_hat = predictimg(face_segment, MODEL)

    if len(np.where(y_hat[0]==1)[0]) == 0: # Fail safe
        continue

    emotion = f'{CLASS_INDICES[np.where(y_hat[0]==1)[0][0]]}'
    # Frame utils
    pos = (face_x, face_y-10)
    cv.putText(frame, emotion, 
        pos, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType
    )

    cv.imshow('cv_recognition_model1', frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
vid.release()
cv.destroyAllWindows()
print('End of script')

1/1 [==============================] - 0s 18ms/step
End of script
